In [5]:
import re
import pandas as pd
import subprocess
import numpy as np

# TQDM 경고 제거를 위한 설정 (다른 import보다 먼저)
import os
os.environ['TQDM_DISABLE'] = '1'

import optuna
from optuna.samplers import TPESampler
import sqlite3
import logging
from datetime import datetime
from typing import Dict, Any
import json
import warnings

# 추가 warning 제거
warnings.filterwarnings('ignore', category=UserWarning, module='tqdm')
warnings.filterwarnings('ignore', message='.*IProgress not found.*')

# Optuna 진행 표시줄 비활성화
optuna.logging.set_verbosity(optuna.logging.WARNING)

# load kinetic data
df_kinetic = pd.read_csv('kinetic_data.csv')
df_mol = pd.read_csv('kinetic_data.csv').filter(like='mol%')
df_time = pd.read_csv('kinetic_data.csv').filter(like='Residence')

class OptunaParameterOptimizer:
    def __init__(self, db_path='optimization_results.db'):
        # define the experimental data
        self.exp_list = ['H2', 'CH4', 'C2H6', 'C2H4', 'C2H2', 'C3H8', 'C3H6', 'C4H10', 'C5H12', 
                        'C', 'CH', 'CH2', 'CH3', 'C2H3', 'C2H5', 'C3H7', 'H',
                        'CH3^+', 'CH4^+', 'CH5^+', 'C2H2^+', 'C2H4^+', 'C2H5^+', 'C2H6^+', 'C3H6^+', 'C3H8^+']
        self.exp_values = [df_mol.iloc[0].tolist() + [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           df_mol.iloc[1].tolist() + [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           df_mol.iloc[2].tolist() + [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           df_mol.iloc[3].tolist() + [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           df_mol.iloc[4].tolist() + [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
        
        # define the file path
        self.kinet_path = 'kinet.inp'
        self.exe_path = 'run_15kV.exe'
        self.db_path = db_path

        # dictionary to save the current parameters
        self.current_parameters = {}
        self.original_parameters = {}
        self.load_current_parameters(init=True)
        self.original_parameters = self.current_parameters.copy()

        # define manipulated variables
        self.max_error = 10000
        self.parameter_range = 0.5  # 매개변수 변화 범위 (±50%)
        self.n_trials = 1000  # 최적화 시도 횟수
        
        # 데이터베이스 초기화
        self.init_database()
    
    def init_database(self):
        """데이터베이스 초기화 및 테이블 생성"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # 매개변수 개수를 동적으로 확인
        param_count = len(self.original_parameters)
        
        # 매개변수 컬럼을 동적으로 생성
        param_columns = []
        for i in range(param_count):
            param_columns.append(f'f{i} REAL')
        
        param_columns_str = ', '.join(param_columns)
        
        # trials 테이블 생성
        create_table_sql = f'''
        CREATE TABLE IF NOT EXISTS trials (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            trial_number INTEGER NOT NULL,
            {param_columns_str},
            error REAL NOT NULL,
            total_time REAL,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
            status TEXT DEFAULT 'COMPLETED',
            notes TEXT
        )
        '''
        
        cursor.execute(create_table_sql)
        
        # 인덱스 생성 (성능 향상)
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_trial_number ON trials(trial_number)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_error ON trials(error)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_timestamp ON trials(timestamp)')
        
        conn.commit()
        conn.close()
        
        print(f"데이터베이스 초기화 완료: {self.db_path}")
    
    def save_trial_to_db(self, trial_number: int, parameters: Dict[str, float], 
                        error: float, total_time: float, status: str = 'COMPLETED', 
                        notes: str = None):
        """trial 결과를 데이터베이스에 저장"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # 매개변수 값들을 순서대로 정렬
        param_values = []
        for i in range(len(self.original_parameters)):
            param_name = f'f{i}'
            param_values.append(parameters.get(param_name, None))
        
        # INSERT SQL 생성
        param_placeholders = ', '.join(['?' for _ in range(len(param_values))])
        insert_sql = f'''
        INSERT INTO trials (trial_number, {', '.join([f'f{i}' for i in range(len(param_values))])}, 
                           error, total_time, status, notes)
        VALUES (?, {param_placeholders}, ?, ?, ?, ?)
        '''
        
        values = [trial_number] + param_values + [error, total_time, status, notes]
        
        cursor.execute(insert_sql, values)
        conn.commit()
        conn.close()
    
    def get_best_trial_from_db(self):
        """데이터베이스에서 최고 성능 trial 조회"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
        SELECT * FROM trials 
        WHERE status = 'COMPLETED' 
        ORDER BY error ASC 
        LIMIT 1
        ''')
        
        result = cursor.fetchone()
        conn.close()
        
        if result:
            columns = [description[0] for description in cursor.description]
            return dict(zip(columns, result))
        return None
    
    def get_trials_summary(self):
        """데이터베이스에서 trial 요약 통계 조회"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
        SELECT 
            COUNT(*) as total_trials,
            MIN(error) as best_error,
            AVG(error) as avg_error,
            COUNT(CASE WHEN status = 'COMPLETED' THEN 1 END) as completed_trials,
            COUNT(CASE WHEN status = 'FAILED' THEN 1 END) as failed_trials
        FROM trials
        ''')
        
        result = cursor.fetchone()
        conn.close()
        
        if result:
            columns = ['total_trials', 'best_error', 'avg_error', 'completed_trials', 'failed_trials']
            return dict(zip(columns, result))
        return None
    
    def export_trials_to_csv(self, filename='trials_export.csv'):
        """데이터베이스 결과를 CSV로 내보내기"""
        conn = sqlite3.connect(self.db_path)
        df = pd.read_sql_query('SELECT * FROM trials ORDER BY trial_number', conn)
        df.to_csv(filename, index=False)
        conn.close()
        print(f"Trial 결과를 {filename}으로 내보냈습니다.")

    def load_current_parameters(self, init=True):
        # 시작인 경우 kinet_ori.inp에서 parameter를 불러옴
        if init:
            with open('kinet_ori.inp', 'r') as f:
                content = f.readlines()
        else:
            # kinet.inp 파일에서 현재 파라미터 값들을 읽어옴
            with open(self.kinet_path, 'r') as f:
                content = f.readlines()

        for line in content:
            if "$ double precision, parameter :: f" in line:
                # f parameter와 값 추출
                pattern = r'f(\d+)\s*=\s*([\d\.\+\-d]+)'
                match = re.search(pattern, line)
                if match:
                    param_name = 'f' + match.group(1)
                    value = float(match.group(2).replace('d','e'))
                    self.current_parameters[param_name] = value

    def modify_parameter(self, param_name: str, new_value: float) -> None:
        # modify the parameter value in the kinet.inp file
        with open(self.kinet_path, 'r') as f:
            content = f.read()
        
        # modify the parameter
        pattern = rf'(parameter :: {param_name} = )([\d\.\+\-d]+)'
        new_value_str = f'{new_value:.4e}'.replace('e', 'd')
        content = re.sub(pattern, f'parameter :: {param_name} = {new_value_str}', content)

        with open(self.kinet_path, 'w') as f:
            f.write(content)
    
    def run_preprocessor(self):
        process = subprocess.Popen(
            './preprocessor.exe',
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=False)

        process.stdin.write(f'{self.kinet_path}\n'.encode())
        process.stdin.flush()
        process.stdin.write('.\n'.encode())
        process.stdin.flush()

        output, error = process.communicate()
        output_str = output.decode('utf-8', errors='ignore') if output else ''
        error_str = error.decode('utf-8', errors='ignore') if error else ''
        
        return output_str, error_str
    
    def compile_zdp(self):
        compile_command = [
            'gfortran', '-o', self.exe_path, 'dvode_f90_m.F90', 'zdplaskin_m.F90', 'run_15kV.F90', 'bolsig_x86_64_g.dll'
        ]
        result = subprocess.run(compile_command, capture_output=True, text=True)
    
        if result.returncode != 0:
            raise Exception(f"{self.exe_path} 컴파일 실패")

    def run_simulation(self):
        try:
            process = subprocess.Popen(
                self.exe_path,    
                stdout=subprocess.PIPE,  
                stderr=subprocess.STDOUT,  
                stdin=subprocess.PIPE,   
                universal_newlines=True,
                bufsize=1,              
            )
            
            while True:
                output = process.stdout.readline()
                
                if not output:
                    break
                    
                if "PRESS ENTER TO EXIT" in output:
                    process.kill()
                    break

                if "WARNING: BOLSIG+ convergence failed" in output:
                    process.stdin.write('\n')
                    process.stdin.flush()

        except:
            pass
        return process
    
    def err_calculation(self):
        # read the result file
        species = []
        with open('qt_species_list.txt', 'r') as f:
            for line in f:
                comp = line[2:]
                species.append(comp.strip())
        
        df_sp = pd.read_csv('qt_densities.txt', sep=r'\s+', header=0, names=['Time [s]']+species)
        
        # calculate the concentration
        H2 = (df_sp['H2'])
        CH4 = (df_sp['CH4'] + df_sp['CH4(V13)'] + df_sp['CH4(V24)'])
        C2H2 = (df_sp['C2H2'] + df_sp['C2H2(V2)'] + df_sp['C2H2(V5)'] + df_sp['C2H2(V13)'])
        C2H4 = (df_sp['C2H4'] + df_sp['C2H4(V1)'] + df_sp['C2H4(V2)'])
        C2H6 = (df_sp['C2H6'] + df_sp['C2H6(V13)'] + df_sp['C2H6(V24)'])
        C3H6 = (df_sp['C3H6'] + df_sp['C3H6(V)'])
        C3H8 = (df_sp['C3H8'] + df_sp['C3H8(V1)'] + df_sp['C3H8(V2)'])
        C4H10 = (df_sp['C4H9H'])
        C5H12 = (df_sp['C5H12'])
        C = (df_sp['C'])
        CH = (df_sp['CH'])
        CH2 = (df_sp['CH2'])
        CH3 = (df_sp['CH3'])
        C2H3 = (df_sp['C2H3'])
        C2H5 = (df_sp['C2H5'])
        C3H7 = (df_sp['C3H7'])
        H = (df_sp['H'])
        CH3_plus = (df_sp['CH3^+'])
        CH4_plus = df_sp['CH4^+']
        CH5_plus = df_sp['CH5^+']
        C2H2_plus = df_sp['C2H2^+']
        C2H4_plus = df_sp['C2H4^+']
        C2H5_plus = df_sp['C2H5^+']
        C2H6_plus = df_sp['C2H6^+']
        C3H6_plus = df_sp['C3H6^+']
        C3H8_plus = df_sp['C3H8^+']

        all_sp = df_sp.sum(axis=1) - df_sp['E']

        t0 = abs(df_sp['Time [s]']-5.654867).argmin()
        t1 = abs(df_sp['Time [s]']-7.270543).argmin()
        t2 = abs(df_sp['Time [s]']-10.17876).argmin()
        t3 = abs(df_sp['Time [s]']-16.9646).argmin()
        t4 = abs(df_sp['Time [s]']-70).argmin()

        sim = []
        for t in [t0, t1, t2, t3, t4]:
            sim_H2 = float(format(H2.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_CH4 = float(format(CH4.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_C2H2 = float(format(C2H2.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_C2H4 = float(format(C2H4.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_C2H6 = float(format(C2H6.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_C3H6 = float(format(C3H6.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_C3H8 = float(format(C3H8.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_C4H10 = float(format(C4H10.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_C5H12 = float(format(C5H12.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_C = float(format(C.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_CH = float(format(CH.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_CH2 = float(format(CH2.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_CH3 = float(format(CH3.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_C2H3 = float(format(C2H3.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_C2H5 = float(format(C2H5.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_C3H7 = float(format(C3H7.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_H = float(format(H.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_CH3_plus = float(format(CH3_plus.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_CH4_plus = float(format(CH4_plus.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_CH5_plus = float(format(CH5_plus.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_C2H2_plus = float(format(C2H2_plus.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_C2H4_plus = float(format(C2H4_plus.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_C2H5_plus = float(format(C2H5_plus.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_C2H6_plus = float(format(C2H6_plus.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_C3H6_plus = float(format(C3H6_plus.iloc[t]/all_sp.iloc[t]*100, '.6f'))
            sim_C3H8_plus = float(format(C3H8_plus.iloc[t]/all_sp.iloc[t]*100, '.6f'))

            out_H2 = sim_H2 - 0.5*sim_CH3 - 0.5*sim_CH3_plus - sim_CH2 - 0.5*sim_C2H3 + 0.5*sim_C2H5 + 0.5*sim_C2H5_plus + 0.5*sim_H + 0.5*sim_C3H7 + 0.5*sim_CH5_plus - 1.5*sim_CH
            out_CH4 = sim_CH4 + sim_CH3 + sim_CH2 + sim_CH + sim_CH5_plus + sim_CH3_plus + sim_CH4_plus
            out_C2H6 = sim_C2H6 + sim_C2H6_plus
            out_C2H4 = sim_C2H4 + sim_C2H4_plus + sim_C2H3 + sim_C2H5 + sim_C2H5_plus
            out_C2H2 = sim_C2H2 + sim_C2H2_plus
            out_C3H8 = sim_C3H8 + sim_C3H8_plus
            out_C3H6 = sim_C3H6 + sim_C3H6_plus + sim_C3H7
            out_C4H10 = sim_C4H10
            out_C5H12 = sim_C5H12
            out_C = sim_C
            newsim = [out_H2, out_CH4, out_C2H6, out_C2H4, out_C2H2, out_C3H8, out_C3H6, out_C4H10, out_C5H12, out_C]
            sim.append(newsim)
            
        w_factor = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
        err = 0
        for i in range(len(self.exp_values)):
            for j in range(len(self.exp_values[i])):
                if j < 10:
                    err += w_factor[j] * ((self.exp_values[i][j] - sim[i][j])/self.exp_values[i][j])**2
                else:
                    err += 0 

        return err, df_sp['Time [s]'].iloc[-1]  

    def objective(self, trial):
        """Optuna 목적함수"""
        try:
            # 각 매개변수에 대해 탐색 범위 설정
            param_values = {}
            for param_name, original_value in self.original_parameters.items():
                # 로그 스케일로 매개변수 탐색 (원본 값의 ±50% 범위)
                low = original_value * (1 - self.parameter_range)
                high = original_value * (1 + self.parameter_range)
                
                # 로그 스케일이 적절한 경우
                if original_value > 0:
                    param_values[param_name] = trial.suggest_float(
                        param_name, low, high, log=True
                    )
                else:
                    param_values[param_name] = trial.suggest_float(
                        param_name, low, high
                    )
            
            # 매개변수 적용
            for param_name, param_value in param_values.items():
                self.modify_parameter(param_name, param_value)
            
            # 시뮬레이션 실행
            self.run_preprocessor()
            self.compile_zdp()
            self.run_simulation()
            
            # 오차 계산
            error, total_time = self.err_calculation()
            
            # 시뮬레이션이 제대로 완료되지 않은 경우 패널티
            status = 'COMPLETED'
            if total_time <= 69.6:
                error = self.max_error
                status = 'FAILED_TIME'
            
            # 데이터베이스에 결과 저장
            self.save_trial_to_db(
                trial_number=trial.number,
                parameters=param_values,
                error=error,
                total_time=total_time,
                status=status,
                notes=f'Time threshold: {total_time:.2f} > 69.6' if total_time > 69.6 else 'Time threshold not met'
            )
            
            print(f'Trial {trial.number}: error = {error:.6f}, time = {total_time:.2f}, status = {status}')
            
            return error
            
        except Exception as e:
            error_msg = str(e)
            print(f'Trial {trial.number} failed: {error_msg}')
            
            # 실패한 trial도 데이터베이스에 저장
            self.save_trial_to_db(
                trial_number=trial.number,
                parameters=param_values if 'param_values' in locals() else {},
                error=self.max_error,
                total_time=0.0,
                status='FAILED_ERROR',
                notes=f'Error: {error_msg}'
            )
            
            return self.max_error

    def optimize(self):
        """Optuna를 사용한 최적화 실행"""
        # SQLite storage를 사용하는 Study 생성
        storage_url = f'sqlite:///{self.db_path}'
        study_name = f'parameter_optimization_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
        
        # TPE 샘플러 사용
        sampler = TPESampler(n_startup_trials=20)
        
        study = optuna.create_study(
            study_name=study_name,
            storage=storage_url,
            direction='minimize', 
            sampler=sampler,
            load_if_exists=True  # 기존 study가 있으면 이어서 진행
        )
        
        # 초기 매개변수로 첫 번째 trial 설정
        print("초기 매개변수 평가 중...")
        initial_params = {}
        for param_name, value in self.original_parameters.items():
            initial_params[param_name] = value
        study.enqueue_trial(initial_params)
        
        # 최적화 실행
        print(f"Optuna 최적화 시작: {self.n_trials}회 시도")
        print(f"Study name: {study_name}")
        print(f"Database: {self.db_path}")
        
        # 진행 표시줄 비활성화하여 최적화 실행
        study.optimize(self.objective, n_trials=self.n_trials, show_progress_bar=False)
        
        # 결과 출력
        print("\n최적화 완료!")
        print(f"최적 오차: {study.best_value:.6f}")
        print("최적 매개변수:")
        for param_name, param_value in study.best_params.items():
            print(f"  {param_name}: {param_value:.6e}")
        
        # 데이터베이스에서 통계 조회
        summary = self.get_trials_summary()
        if summary:
            print(f"\nTrial 통계:")
            print(f"  총 시도: {summary['total_trials']}")
            print(f"  성공: {summary['completed_trials']}")
            print(f"  실패: {summary['failed_trials']}")
            print(f"  최고 오차: {summary['best_error']:.6f}")
            print(f"  평균 오차: {summary['avg_error']:.6f}")
        
        # 최적 매개변수를 파일에 적용
        for param_name, param_value in study.best_params.items():
            self.modify_parameter(param_name, param_value)
        
        return study
    
    def resume_optimization(self, study_name: str, additional_trials: int = None):
        """기존 study를 이어서 최적화 실행"""
        storage_url = f'sqlite:///{self.db_path}'
        
        if additional_trials is None:
            additional_trials = self.n_trials
        
        try:
            study = optuna.load_study(
                study_name=study_name,
                storage=storage_url
            )
            
            print(f"기존 study 로드됨: {study_name}")
            print(f"현재까지 {len(study.trials)}개 trial 완료")
            print(f"추가로 {additional_trials}개 trial 실행")
            
            study.optimize(self.objective, n_trials=additional_trials, show_progress_bar=False)
            
            print(f"\n최적화 재개 완료!")
            print(f"총 trial 수: {len(study.trials)}")
            print(f"최적 오차: {study.best_value:.6f}")
            
            return study
            
        except KeyError:
            print(f"Study '{study_name}'을 찾을 수 없습니다.")
            print("사용 가능한 study 목록:")
            self.list_studies()
            return None
    
    def list_studies(self):
        """데이터베이스에 저장된 모든 study 목록 조회"""
        storage_url = f'sqlite:///{self.db_path}'
        
        try:
            studies = optuna.get_all_study_summaries(storage=storage_url)
            if studies:
                print("저장된 Study 목록:")
                for study in studies:
                    print(f"  - {study.study_name}: {study.n_trials} trials")
            else:
                print("저장된 study가 없습니다.")
        except Exception as e:
            print(f"Study 목록 조회 실패: {e}")
    
    def analyze_parameters(self, top_n: int = 10):
        """상위 N개 trial의 매개변수 분석"""
        conn = sqlite3.connect(self.db_path)
        
        # 상위 N개 trial 조회
        query = f'''
        SELECT * FROM trials 
        WHERE status = 'COMPLETED' 
        ORDER BY error ASC 
        LIMIT {top_n}
        '''
        
        df = pd.read_sql_query(query, conn)
        conn.close()
        
        if df.empty:
            print("분석할 완료된 trial이 없습니다.")
            return None
        
        print(f"\n상위 {len(df)}개 trial 매개변수 분석:")
        print("=" * 60)
        
        # 매개변수 컬럼만 선택
        param_cols = [col for col in df.columns if col.startswith('f')]
        
        for i, row in df.iterrows():
            print(f"Trial {int(row['trial_number'])} (Error: {row['error']:.6f}):")
            for param in param_cols:
                if pd.notna(row[param]):
                    print(f"  {param}: {row[param]:.6e}")
            print("-" * 40)
        
        # 매개변수 통계
        print("\n매개변수 통계 (상위 trials):")
        param_stats = df[param_cols].describe()
        print(param_stats)
        
        return df
    
    def plot_optimization_progress(self, output_file: str = 'optimization_progress.html'):
        """최적화 진행 상황 시각화"""
        try:
            import plotly.graph_objects as go
            import plotly.express as px
            from plotly.subplots import make_subplots
            
            conn = sqlite3.connect(self.db_path)
            df = pd.read_sql_query('''
                SELECT trial_number, error, timestamp, status
                FROM trials 
                ORDER BY trial_number
            ''', conn)
            conn.close()
            
            if df.empty:
                print("시각화할 데이터가 없습니다.")
                return
            
            # 완료된 trial만 필터링
            completed_df = df[df['status'] == 'COMPLETED'].copy()
            
            if completed_df.empty:
                print("완료된 trial이 없습니다.")
                return
            
            # 누적 최소값 계산
            completed_df['best_so_far'] = completed_df['error'].cummin()
            
            # 서브플롯 생성
            fig = make_subplots(
                rows=2, cols=2,
                subplot_titles=('최적화 진행 상황', 'Error 분포', 'Trial별 Error', 'Status 분포'),
                specs=[[{"secondary_y": False}, {"secondary_y": False}],
                       [{"secondary_y": False}, {"secondary_y": False}]]
            )
            
            # 1. 최적화 진행 상황
            fig.add_trace(
                go.Scatter(x=completed_df['trial_number'], y=completed_df['best_so_far'],
                          mode='lines', name='Best Error So Far'),
                row=1, col=1
            )
            
            # 2. Error 분포 히스토그램
            fig.add_trace(
                go.Histogram(x=completed_df['error'], nbinsx=30, name='Error Distribution'),
                row=1, col=2
            )
            
            # 3. Trial별 Error 산점도
            fig.add_trace(
                go.Scatter(x=completed_df['trial_number'], y=completed_df['error'],
                          mode='markers', name='Trial Errors'),
                row=2, col=1
            )
            
            # 4. Status 분포
            status_counts = df['status'].value_counts()
            fig.add_trace(
                go.Bar(x=status_counts.index, y=status_counts.values, name='Status Count'),
                row=2, col=2
            )
            
            fig.update_layout(height=800, showlegend=True, 
                            title_text="매개변수 최적화 분석")
            fig.write_html(output_file)
            print(f"시각화 결과가 {output_file}에 저장되었습니다.")
            
        except ImportError:
            print("시각화를 위해 plotly를 설치하세요: pip install plotly")
        except Exception as e:
            print(f"시각화 생성 실패: {e}")

if __name__ == "__main__":
    optimizer = OptunaParameterOptimizer(db_path='optimization_results.db')
    
    # 명령행 인자 처리 (간단한 예시)
    import sys
    
    if len(sys.argv) > 1:
        command = sys.argv[1]
        
        if command == 'optimize':
            # 새로운 최적화 시작
            study = optimizer.optimize()
            
        elif command == 'resume':
            if len(sys.argv) > 2:
                study_name = sys.argv[2]
                additional_trials = int(sys.argv[3]) if len(sys.argv) > 3 else None
                study = optimizer.resume_optimization(study_name, additional_trials)
            else:
                print("사용법: python script.py resume <study_name> [additional_trials]")
                
        elif command == 'list':
            # Study 목록 조회
            optimizer.list_studies()
            
        elif command == 'analyze':
            # 매개변수 분석
            top_n = int(sys.argv[2]) if len(sys.argv) > 2 else 10
            optimizer.analyze_parameters(top_n)
            
        elif command == 'plot':
            # 시각화
            optimizer.plot_optimization_progress()
            
        elif command == 'export':
            # CSV 내보내기
            filename = sys.argv[2] if len(sys.argv) > 2 else 'trials_export.csv'
            optimizer.export_trials_to_csv(filename)
            
        elif command == 'summary':
            # 통계 요약
            summary = optimizer.get_trials_summary()
            if summary:
                print("최적화 통계:")
                for key, value in summary.items():
                    print(f"  {key}: {value}")
            
        else:
            print("사용 가능한 명령: optimize, resume, list, analyze, plot, export, summary")
    else:
        # 기본: 새로운 최적화 시작
        study = optimizer.optimize()
        
        # 최적화 완료 후 분석
        print("\n=== 최적화 완료 후 분석 ===")
        optimizer.analyze_parameters(5)
        optimizer.plot_optimization_progress()
        optimizer.export_trials_to_csv()

데이터베이스 초기화 완료: optimization_results.db
사용 가능한 명령: optimize, resume, list, analyze, plot, export, summary
